# Implementation and evaluation of K-nearest neighbors (KNN) algorithm for handwritten digit recognition
### Data Analysis MoBi SoSe2022, Topic 01: Biomedical Image Analysis
### Tutorin: Marie Becker
### Team 02: Lena Fleischhacker, Pia Röhrich, Hellen Röttgen, Benjamin Wehnert
#### July 2022

# Abstract

# Table of contents
1. Introduction
2. Material
    i. Imports
    ii. average images
3. Methods
    i. Z-tranformation
    ii. Principal component analysis
    iii. k-nearest neighbors
4. Results
    i. Z-tranformation
    ii. Principal component analysis
    iii. k-nearest neighbors
 5. Discussion  
 6. Biblliography 



# 1) Introduction
The main goal of the project is to write an algorithm that accurately recognizes handwritten digits from the MNIST dataset using the K-nearest neighbors method. Digit recognition is a form of pattern recognition which describes the operation of identifying digits from images. This is not an easy task, taking into consideration that handwritten numbers are not written in the same style. Thus, the categories are harder to classify. With almost all processes in everyday life being digitalized, e.g. banking transactions or contact forms, developing such algorithms has become extremely useful and almost inevitable.

For this reason, the MNIST data set has been well studied and analyzed over the past decades, leading to the development of algorithms that recognize digits with the same accuracy as human beings. This laid the foundation for the development of more advanced pattern recognition algorithms such as detecting house numbers from photos with the SVHN (Street View house numbers) dataset. These algorithms can be very useful for future applications like better navigation systems.

However, in order to properly understand the fundmentals of machine learning, it is useful to reproduce the basic steps of digit recognition using the MNIST data set.

Additionally, an analysis of the accuracy of recognition with the KNN method is going to be part of the project.

In the last part of the project, we looked for a creative application of the data set


# 2) Material (information data set)??
The data set that we used for the project is the standardized MNIST data set. MNIST stands for "Modified National Institute of Standard and Technology database". The data set consists of 70000 images which are divided into a training data set comprising 60000 images and a test data set comprising 10000 images. Each image contains a handwritten digit ranging from zero to nine. 

The images are stored as CSV (comma-seperated values) files. Each image consists of 28 x 28 = 784 pixels with intensity values ranging from 0 to 255. This means that all training images can be displayed in a 60000 x 785 matrix with the first column giving the actual number displayed in the image. Consequently, the test images can be dispalyed as a 10000 x 784 matrix.


## 2.i.) Imports
To implement 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import Functions.PCA as pca
import Functions.data_load as dat
import Functions.visualization as vis
import Functions.average_img as avg

In [ ]:
#codeprinting normal numbers

## 2.ii.) Average images
To get a first impression of the images in the data set, we wrote an algorithm calculating mean digits. 
For this, we used all images from the training data set containing the same number. We then summed up the intensitied for each pixel an divided that values by the number of images used. Surprisingly, the numbers were still recognizable.  


In [ ]:
#code printing average images, Bilder zum Vergleich

# 3) Methods

## 3.i.) Z-Tranformation (PCA preparation)
First of all, the data is z-transformed for easier handling. Using the z-transformation, values of the sample are converted to z-scores. The distribution of these z-scores has a mean of 0 and a standard deviation of 1. Z-scores make the sample comparable by making the data non-dimensional. The z-scores are calculated as follows:

$$ Z = \frac{\left(X_i-\bar{X}\right)}{\sigma_{x}} $$

Applied to the project, this means that the average intensity and standard deviation for each pixel need to be calculated. After that, each pixel in each of the images needs to be z-transformed.

In [ ]:
#code z-transformation (one line)

## 3.ii.) Principal Component Analysis
Prior to the implementation of Knn, a Principal Component Analysis (PCA) is implemented to minimize computing power by reducing dimensionality. Knn is a lazy learning algorithm, meaning that instead of learning from the training data, the algorithm simply compares input with the trainig data each time it runs. Consequently, a dimension reduction shortens the run time.

However, despite losing dimensions, hardly any information should be lost. Therefore, the first step is to identify the correlations between different features of the dataset (features = intensity values) and putting those values into a correlation matrix. This is done using Pearson correlation which is calcualted as follows:

$$ Corr(x,y) = \frac{1}{{N-1}}\sum_{i=1}^N\frac{(x_i-\overline{x})}{s_x}\frac{(y_i-\overline{y})}{s_y}\ $$

In the project, just the covariance was calculated as the division by the standard deviation becomes unnecessary due to the the previous z-transformation.

After this step, the eigenvectors and eigenvalues of the covariance matrix are calculated. Eigenvectors v of a matrix A are those v that, multiplied with A, result in a vector in the same or opposite direction. The multiplication of v with A equals the multiplication of v with a certain numeric value, also called eigenvalue λ. To sum up, eigenvectors and eigenvalues satisfy the following equation:

$$ Av_i=λ_iv_i $$

If calculated by hand, the eigenvalues need to be calculated first in order to calcuate the eigenvectors. In the project, this step was replaced by simply using the eigenvector function integrated in NumPy. The eigenvectors are then put into a new matrix and sorted according to their corresponding eingenvalues. The higher the eigenvalue, the higher is the amount of variance covered by one principal component. Spatially, this step describes the rotation of the 784-dimensional correlation matrix of the insensity values of the training images. this allows us to look at the data points from a different angle, defining new axes called principal components. 

From the matrix with the eigenvalues, a certain number of principal components is chosen. The ideal number of principle components can be found by ?

The last step of a PCA is the transformation of the original matrix with the principle components. This is done by multiplying A with a matrix made up of the top k eigenvectors, resulting in the transformed data matrix.

In summary, the principal components allow us to reduce dimensions and run time by leaving out highly correlated information.


### 2.2.1.) Visualization of PCA


In [ ]:
#code PCA visualization

## 3.iii.) KNN
K-nearest neighbors (KNN) is the machine learning algorithm that we are using to classify the numbers. It is a simple, supervised machine learning algorithm.

## 3. iv.) methods used for creative application

# 4) Results

## result creative application

## 4.i.) Results Z-Transformation

## 4.ii.) Results PCA

## 4.iii.) Results kNN

# 5) Discussion

# 6) Bibliography
Netzer, Y. et al. "Reading Digits in Natural Images with Unsupervised Feature Learning." Proceedings of the Workshop on Neural Information Processing Systems (2011)